# Detrended Fluctuation Analysis

https://github.com/CSchoel/nolds/blob/master/nolds/measures.py

In [1]:
import numpy as np
import numba
import warnings
from entropy import utils

from nolds import dfa

warnings.simplefilter('ignore', np.RankWarning)

np.random.seed(1234567)
x = np.random.rand(1000)

In [2]:
def logarithmic_n(min_n, max_n, factor):
    """
    Creates a list of values by successively multiplying a minimum value min_n by
    a factor > 1 until a maximum value max_n is reached.
    Non-integer results are rounded down.
    Args:
    min_n (float):
      minimum value (must be < max_n)
    max_n (float):
      maximum value (must be > min_n)
    factor (float):
      factor used to increase min_n (must be > 1)
    Returns:
    list of integers:
      min_n, min_n * factor, min_n * factor^2, ... min_n * factor^i < max_n
      without duplicates
    """
    assert max_n > min_n
    assert factor > 1
    # stop condition: min * f^x = max
    # => f^x = max/min
    # => x = log(max/min) / log(f)
    max_i = int(np.floor(np.log(1.0 * max_n / min_n) / np.log(factor)))
    ns = [min_n]
    for i in range(max_i + 1):
        n = int(np.floor(min_n * (factor ** i)))
        if n > ns[-1]:
            ns.append(n)
    return ns

In [3]:
def dfa2(x, nvals=None, overlap=False):
    """
    Detrended fluctuation analysis (DFA).
    
    Parameters
    ----------
    x : list or np.array
        One-dimensional time series.
    nvals : list or np.array of int
        subseries sizes at which to calculate fluctuation
        (default: logarithmic_n(4, 0.1*len(data), 1.2))
    overlap (boolean):
        if True, the windows W_(n,i) will have a 50% overlap,
        otherwise non-overlapping windows will be used
    
    Returns
    -------
    dfa : float
        the estimate alpha for the Hurst parameter (alpha < 1: stationary
        process similar to fractional Gaussian noise with H = alpha,
        alpha > 1: non-stationary process similar to fractional Brownian
        motion with H = alpha - 1)
        
    Notes
    -----
    Detrended fluctuation analysis, much like the Hurst exponent, is used to
    find long-term statistical dependencies in time series.
    
    The idea behind DFA originates from the definition of self-affine
    processes. A process X is said to be self-affine if the standard deviation
    of the values within a window of length n changes with the window length
    factor L in a power law:
    
    .. math:: std(X, L * n) = L^H * std(X, n)
    
    where std(X, k) is the standard deviation of the process X calculated over
    windows of size k. In this equation, H is called the Hurst parameter, which
    behaves indeed very similar to the Hurst exponent.
    
    Like the Hurst exponent, H can be obtained from a time series by
    calculating std(X,n) for different n and fitting a straight line to the
    plot of log(std(X,n)) versus log(n).
    
    To calculate a single std(X,n), the time series is split into windows of
    equal length n, so that the ith window of this size has the form
    
    .. math:: W_(n,i) = [x_i, x_(i+1), x_(i+2), ... x_(i+n-1)]
    
    The value std(X,n) is then obtained by calculating std(W_(n,i)) for each i
    and averaging the obtained values over i.
    
    The aforementioned definition of self-affinity, however, assumes that the
    process is  non-stationary (i.e. that the standard deviation changes over
    time) and it is highly influenced by local and global trends of the time
    series.
    
    To overcome these problems, an estimate alpha of H is calculated by using a
    "walk" or "signal profile" instead of the raw time series. This walk is
    obtained by substracting the mean and then taking the cumulative sum of the
    original time series. The local trends are removed for each window
    separately by fitting a polynomial p_(n,i) to the window W_(n,i) and then
    calculating W'_(n,i) = W_(n,i) - p_(n,i) (element-wise substraction).
    We then calculate std(X,n) as before only using the "detrended" window
    W'_(n,i) instead of W_(n,i). Instead of H we obtain the parameter alpha
    from the line fitting.
    
    For alpha < 1 the underlying process is stationary and can be modelled as
    fractional Gaussian noise with H = alpha. This means for alpha = 0.5 we
    have no correlation or "memory", for 0.5 < alpha < 1 we have a memory with
    positive correlation and for alpha < 0.5 the correlation is negative.
    For alpha > 1 the underlying process is non-stationary and can be modeled
    as fractional Brownian motion with H = alpha - 1.
    
    THE CODE AND DOCUMENTATION ARE ORIGINALLY FROM THE `NOLDS` PYTHON PACKAGE.
    
    References
    ----------
    .. [dfa_1] C.-K. Peng, S. V. Buldyrev, S. Havlin, M. Simons,
               H. E. Stanley, and A. L. Goldberger, “Mosaic organization of
               DNA nucleotides,” Physical Review E, vol. 49, no. 2, 1994.
    .. [dfa_2] R. Hardstone, S.-S. Poil, G. Schiavone, R. Jansen,
               V. V. Nikulin, H. D. Mansvelder, and K. Linkenkaer-Hansen,
               “Detrended fluctuation analysis: A scale-free view on neuronal
               oscillations,” Frontiers in Physiology, vol. 30, 2012.
               
    
    """
    x = np.array(x)
    N = x.size
    
    if nvals is None:
        nvals = np.array(logarithmic_n(4, 0.1 * N, 1.2))
    
    # create the signal profile
    # (cumulative sum of deviations from the mean => "walk")
    walk = np.cumsum(x - x.mean())
    
    fluctuations = np.zeros(nvals.size)
    
    for i_n, n in enumerate(nvals):
        # subdivide data into chunks of size n
        if overlap:
            # step size n/2 instead of n
            d = np.array([walk[i:i + n] for i in range(0, len(walk) - n, n // 2)])
        else:
            # non-overlapping windows => we can simply do a reshape
            d = walk[:N - (N % n)]
            d = d.reshape((N // n, n))
        # calculate local trends as polynomes
        ran_n = np.arange(n, dtype=np.float64)
        ran_n_mean = ran_n.mean()
        d_len = d.shape[0]
        d_mean = d.mean(1)
        slope = np.empty(d_len)
        intercept = np.empty(d_len)
        trend = np.empty((d_len, ran_n.size))
        for i in range(d_len):
            slope[i] = utils._slope_lstsq(ran_n, d[i])
            intercept[i] = d_mean[i] - slope[i] * ran_n_mean
            trend[i, :] = np.polyval([slope[i], intercept[i]], ran_n)
            
        # calculate standard deviation ("fluctuation") of walks in d around trend
        flucs = np.sqrt(np.sum((d - trend) ** 2, axis=1) / n)
        # calculate mean fluctuation over all subsequences
        fluctuations[i_n] = flucs.sum() / flucs.size

    # filter zeros from fluctuations
    nonzero = np.nonzero(fluctuations)[0]
    fluctuations = fluctuations[nonzero]
    nvals = nvals[nonzero]

    if len(fluctuations) == 0:
        # all fluctuations are zero => we cannot fit a line
        slope = np.nan
    else:
        slope = utils._slope_lstsq(np.log(nvals), np.log(fluctuations))
    return slope

In [4]:
print(dfa(x, overlap=False))
print(dfa2(x, overlap=False))

0.5330608987017387
0.5330608987017413


In [5]:
print(dfa(x, overlap=True))
print(dfa2(x, overlap=True))

0.5347468094425519
0.5347468094425555


In [6]:
%timeit dfa(x, overlap=False)
%timeit dfa2(x)

82 ms ± 753 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
14.6 ms ± 233 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Numba

In [11]:
from math import floor, log

@numba.jit('i8[:](f8, f8, f8)', nopython=True)
def log_n(min_n, max_n, factor):
    max_i = int(floor(log(1.0 * max_n / min_n) / log(factor)))
    ns = [min_n]
    for i in range(max_i + 1):
        n = int(floor(min_n * (factor ** i)))
        if n > ns[-1]:
            ns.append(n)
    return np.array(ns, dtype=np.int64)

@numba.jit('f8(f8[:])', nopython=True)
def dfa3(x):
    N = len(x)
    nvals = log_n(4, 0.1 * N, 1.2)
    walk = np.cumsum(x - x.mean())
    fluctuations = np.zeros(len(nvals))
    
    for i_n, n in enumerate(nvals):
        d = np.reshape(walk[:N - (N % n)], (N // n, n))
        ran_n = np.array([float(na) for na in range(n)])
        ran_n_mean = ran_n.mean()
        d_len = len(d)
        slope = np.empty(d_len)
        intercept = np.empty(d_len)
        trend = np.empty((d_len, ran_n.size))
        for i in range(d_len):
            sl = utils._slope_lstsq(ran_n, d[i])
            di_mean = d[i].mean()
            inter = di_mean - sl * ran_n_mean
            slope[i] = sl
            intercept[i] = inter
            y = np.zeros_like(ran_n)
            # Equivalent to np.polyval function
            for p in [sl, inter]:
                y = y * ran_n + p
            trend[i, :] = y

        # calculate standard deviation ("fluctuation") of walks in d around trend
        flucs = np.sqrt(np.sum((d - trend) ** 2, axis=1) / n)
        # calculate mean fluctuation over all subsequences
        fluctuations[i_n] = flucs.sum() / flucs.size
      
    # Filter zero
    nonzero = np.nonzero(fluctuations)[0]
    fluctuations = fluctuations[nonzero]
    nvals = nvals[nonzero]

    if len(fluctuations) == 0:
        # all fluctuations are zero => we cannot fit a line
        dfa = np.nan
    else:
        dfa = utils._slope_lstsq(np.log(nvals), np.log(fluctuations))
    return dfa   

In [12]:
print(dfa(x, overlap=False))
print(dfa2(x))
print(dfa3(x))

0.5330608987017387
0.5330608987017413
0.5330608987017409


In [13]:
%timeit dfa(x, overlap=False)
%timeit dfa2(x, overlap=False)
%timeit dfa3(x)

89.6 ms ± 4.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
16 ms ± 335 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
800 µs ± 27.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
np.random.seed(1234567)
RANDOM_TS = np.random.rand(3000)
SF_TS = 100
PURE_SINE = np.sin(2 * np.pi * 1 * np.arange(3000) / 100)

print(dfa(PURE_SINE, overlap=False))
print(dfa2(PURE_SINE))
print(dfa3(PURE_SINE))

1.615840712681188
1.6158407126811893
1.6158407126811898
